<a href="https://colab.research.google.com/github/yngkyng/first_project-1/blob/master/5_5_%EC%84%9C%EC%9A%B8%EA%B2%BD%EC%B0%B0%EC%84%9C_%EC%A7%80%EB%8F%84%ED%91%9C%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Selenium > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
import math

In [3]:
chrome_options = webdriver.ChromeOptions()             # 웹 드라이버 옵션 지정
chrome_options.add_argument('--headless')              # 웹 브라우저 띄우지 않음
chrome_options.add_argument('--no-sandbox')            # 별도 웹페이지 작동 해제
chrome_options.add_argument('--disable-dev-shm-usage') # 공유 안함
chrome_options.add_argument('--single-process')

#서울지역 경찰서 정보 추출

In [ ]:
driver = webdriver.Chrome('chromedriver', options=chrome_options) # 웹 드리이버 객체 생성
url = 'https://www.police.go.kr/user/bbs/BD_selectBbsList.do?q_bbsCode=1038&q_tab=1'  
driver.get(url)
time.sleep(3)

#정보 추출하기
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.table tr')
    #trs = driver.find_elements(By. CSS_SELECTOR, '.table tr')
lines=[]
for tr in trs:
    try:
        title = tr.select('.show-col1')[0].text.strip()
        address = tr.select('.show-col1')[1].text.strip()
    except:
        continue
    lines.append([title, address])

df= pd.DataFrame(lines, columns=['title','address'])
df.head()

In [ ]:
df.to_csv('서울 경찰서 정보.csv', index=False)

#카카오API로 위도경도 추가

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

In [ ]:
with open(filename) as f:
    api_key = f.read()

import requests
from urllib.parse import quote

In [ ]:
df = pd.read_csv('/content/서울 경찰서 정보.csv')

In [ ]:
lng_list = []
lat_list = []

for i in df.index:
    addr = df.address[i]
    search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    url = f'{search_url}?query={quote(addr)}'
    result = requests.get(url, headers={'Authorization': f'KakaoAK {api_key}'}).json()
    try:
        lng = float(result['documents'][0]['address']['x'])
        lat = float(result['documents'][0]['address']['y'])
    except:
        print(df.title[i])
    lng_list.append(lng)
    lat_list.append(lat)

In [ ]:
df['위도'] = lat_list
df['경도'] = lng_list

#df.drop(df[df['title']=='은평소방서-수색-119 안전센터'].index, inplace=True)

df.to_csv('서울경찰서.csv', index=False)

#시각화

In [ ]:
import folium

In [ ]:
seoul_map = folium.Map(location=[37.55, 126.98], zoom_start=12)
df= pd.read_csv('/content/서울경찰서.csv')
for i, lat, lng in zip(df.index, df.위도, df.경도):
    folium.Marker([lat,lng], 
                  popup= folium.Popup(f'{df.title[i]} / {df.address[i]}', max_width=200),
                  tooltip= df.title[i]
                  ).add_to(seoul_map)
title = '<h3 align="center" style="font-size:20px; font-weight:bold">서울 소재 경찰서 위치</h3>'
seoul_map.get_root().html.add_child(folium.Element(title))
seoul_map